In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window
spark = SparkSession.builder.master("local[*]").appName("windowing").getOrCreate()

In [0]:
data = [
    ["000", "107", "Emily Lee", 26, "", 46000, "2019-01-01"],
    ["001", "101", "John Doe", 30, "Male", 50000, "2015-01-01"],
    ["002", "101", "Jane Smith", 25, "Female", 45000, "2016-02-15"],
    ["003", "102", "Bob Brown", 35, "Male", 55000, "2014-05-01"],
    ["004", "102", "Alice Lee", 28, "Female", 48000, "2017-09-30"],
    ["005", "103", "Jack Chan", 40, "Male", 60000, "2013-04-01"],
    ["006", "103", "Jill Wong", 32, "Female", 52000, "2018-07-01"],
    ["007", "101", "James Johnson", 42, "Male", 70000, "2012-03-15"],
    ["008", "102", "Kate Kim", 29, "Female", 51000, "2019-10-01"],
    ["009", "103", "Tom Tan", 33, "Male", 58000, "2016-06-01"],
    ["010", "104", "Lisa Lee", 27, "Female", 47000, "2018-08-01"],
    ["011", "104", "David Park", 38, "Male", 65000, "2015-11-01"],
    ["012", "105", "Susan Chen", 31, "Female", 54000, "2017-02-15"],
    ["013", "106", "Brian Kim", 45, "Male", 75000, "2011-07-01"],
    ["014", "107", "Emily Lee", 26, "Female", 46000, "2019-01-01"],
    ["015", "106", "Michael Lee", 37, "Male", 63000, "2014-09-30"],
    ["016", "107", "Kelly Zhang", 30, "Female", 49000, "2018-04-01"],
    ["017", "105", "George Wang", 34, "Male", 57000, "2016-03-15"],
    ["018", "104", "Nancy Liu", 29, "Female", 50000, "2017-06-01"],
    ["019", "103", "Steven Chen", 36, "Male", 62000, "2015-08-01"],
    ["020", "102", "Grace Kim", 32, "Female", 53000, "2018-11-01"],
]

emp_schema = "employee_id string, department_id string, name string, age string, gender string, salary string, hire_date string"

In [0]:
df = spark.createDataFrame(data,schema = emp_schema)

In [0]:
df.show()

+-----------+-------------+-------------+---+------+------+----------+
|employee_id|department_id|         name|age|gender|salary| hire_date|
+-----------+-------------+-------------+---+------+------+----------+
|        000|          107|    Emily Lee| 26|      | 46000|2019-01-01|
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15|
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|
|        005|          103|    Jack Chan| 40|  Male| 60000|2013-04-01|
|        006|          103|    Jill Wong| 32|Female| 52000|2018-07-01|
|        007|          101|James Johnson| 42|  Male| 70000|2012-03-15|
|        008|          102|     Kate Kim| 29|Female| 51000|2019-10-01|
|        009|          103|      Tom Tan| 33|  Male| 58000|2016-06-01|
|        010|          104|     Lisa Lee| 27|Female| 47000|2018-08-01|
|     

# Need to use distinct() function to get the non-duplicate values

In [0]:
df_non_duplicate = df.distinct()
df_non_duplicate.show()

+-----------+-------------+-------------+---+------+------+----------+
|employee_id|department_id|         name|age|gender|salary| hire_date|
+-----------+-------------+-------------+---+------+------+----------+
|        000|          107|    Emily Lee| 26|      | 46000|2019-01-01|
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15|
|        007|          101|James Johnson| 42|  Male| 70000|2012-03-15|
|        005|          103|    Jack Chan| 40|  Male| 60000|2013-04-01|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|
|        006|          103|    Jill Wong| 32|Female| 52000|2018-07-01|
|        009|          103|      Tom Tan| 33|  Male| 58000|2016-06-01|
|        008|          102|     Kate Kim| 29|Female| 51000|2019-10-01|
|        011|          104|   David Park| 38|  Male| 65000|2015-11-01|
|     

## Selecting unique department_id

In [0]:
emp_distinct_department_id = df.select("department_id").distinct().show()

+-------------+
|department_id|
+-------------+
|          101|
|          107|
|          102|
|          103|
|          104|
|          106|
|          105|
+-------------+



# Window function---> provides partitioning on group of data

In [0]:
# select max salary over partition

window_spec = Window.partitionBy(col("department_id")).orderBy(col("salary").desc())
max_final = df.withColumn("max_salary",max(col("salary")).over(window_spec)).show()



+-----------+-------------+-------------+---+------+------+----------+----------+
|employee_id|department_id|         name|age|gender|salary| hire_date|max_salary|
+-----------+-------------+-------------+---+------+------+----------+----------+
|        007|          101|James Johnson| 42|  Male| 70000|2012-03-15|     70000|
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|     70000|
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15|     70000|
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|     55000|
|        020|          102|    Grace Kim| 32|Female| 53000|2018-11-01|     55000|
|        008|          102|     Kate Kim| 29|Female| 51000|2019-10-01|     55000|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|     55000|
|        019|          103|  Steven Chen| 36|  Male| 62000|2015-08-01|     62000|
|        005|          103|    Jack Chan| 40|  Male| 60000|2013-04-01|     62000|
|        009|   

## 2nd highest salary in dept

In [0]:
window_df = Window.partitionBy("department_id").orderBy(col("salary").desc())

row_df = df.withColumn("row_number", row_number().over(window_df)).where("row_number =2").show()


# we can also write in SQL also if you find Pyspark API is difficult.

row_df_2 = df.withColumn("rn", expr("row_number() over(partition by department_id order by salary desc)")).where("rn = 2").show()

+-----------+-------------+-----------+---+------+------+----------+----------+
|employee_id|department_id|       name|age|gender|salary| hire_date|row_number|
+-----------+-------------+-----------+---+------+------+----------+----------+
|        001|          101|   John Doe| 30|  Male| 50000|2015-01-01|         2|
|        020|          102|  Grace Kim| 32|Female| 53000|2018-11-01|         2|
|        005|          103|  Jack Chan| 40|  Male| 60000|2013-04-01|         2|
|        018|          104|  Nancy Liu| 29|Female| 50000|2017-06-01|         2|
|        012|          105| Susan Chen| 31|Female| 54000|2017-02-15|         2|
|        015|          106|Michael Lee| 37|  Male| 63000|2014-09-30|         2|
|        000|          107|  Emily Lee| 26|      | 46000|2019-01-01|         2|
+-----------+-------------+-----------+---+------+------+----------+----------+

+-----------+-------------+-----------+---+------+------+----------+---+
|employee_id|department_id|       name|age|gen